<a href="https://colab.research.google.com/github/kurtsenol/pandas/blob/master/WebScraping/webscraping_glass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WEB SCRAPING

In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'
# chrome_options.add_argument(f'user-agent={user_agent}')

wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

     |████████████████████████████████| 911kB 2.8MB/s 
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Get:4 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [890 kB]
Get:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: use options instead of chrome_options
  app.launch_new_instance()


I use the specific URL below not to have copyright problem. Otherwise we can use https://google.com  as search URL .

In [ ]:
#Specify Search URL 
search_url = "https://www.google.com/search?q={q}&tbm=isch&tbs=sur%3Afc&hl=en&ved=0CAIQpwVqFwoTCKCa1c6s4-oCFQAAAAAdAAAAABAC&biw=1251&bih=568"
wd.get(search_url.format(q='wear glasses'))

In [ ]:
open('source.html', 'w').write(wd.page_source)

937401

In [ ]:
wd.title

'wear glasses - Google Search'

In [ ]:
import time

#Scroll to the end of the page

def scroll_to_end(driver):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)#sleep_between_interactions

In [ ]:
from selenium.common.exceptions import ElementClickInterceptedException

#no license issues

def getImageUrls(name,totalImgs,driver):
    
    search_url = "https://www.google.com/search?q={q}&tbm=isch&tbs=sur%3Afc&hl=en&ved=0CAIQpwVqFwoTCKCa1c6s4-oCFQAAAAAdAAAAABAC&biw=1251&bih=568"
    driver.get(search_url.format(q=name))

    img_urls = set()
    img_count = 0
    results_start = 0  
    
    while(img_count<totalImgs): #Extract actual images now
        
        scroll_to_end(driver)
        
        thumbnail_results = driver.find_elements_by_xpath("//img[contains(@class,'Q4LuWd')]")
        totalResults=len(thumbnail_results)
        print(f"Found: {totalResults} search results. Extracting links from {results_start}:{totalResults}")
        
        for i in range(results_start, totalResults):
            
            img = thumbnail_results[i]

            try:

                img.click()
                time.sleep(2)
                
                actual_images = wd.find_elements_by_xpath("//img[@data-noaft='1']")
                
                for actual_image in actual_images:
                    img_link = actual_image.get_attribute('src')
                    if img_link and 'https' in img_link:
                        img_urls.add(img_link)
            
            # actual_images = driver.find_elements_by_css_selector('img.n3VNCb')
            # for actual_image in actual_images:
            #     if actual_image.get_attribute('src') and 'https' in actual_image.get_attribute('src'):
            #         img_urls.add(actual_image.get_attribute('src'))

            except ElementClickInterceptedException as err:
                print(err)
            
            img_count=len(img_urls)
            
            # check whether get enough images
            if img_count >= totalImgs:
                print(f"Found: {img_count} image links")
                break

            # check whether we need to load more images on the web page                 
            elif i== (totalResults-1):
                load_more_button = driver.find_element_by_css_selector(".mye4qd")
                driver.execute_script("document.querySelector('.mye4qd').click();")
                results_start = len(thumbnail_results)
                print("load more clicked")
                time.sleep(5)
            else:
                print("Found:", img_count, "looking for more image links ...")                
                
                
                
    return img_urls

Download & save each image in Destination directory

In [ ]:
import requests
from PIL import Image

def downloadImages(folder_path,file_name,url):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - COULD NOT DOWNLOAD {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        
        file_path = os.path.join(folder_path, file_name)
        
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SAVED - {url} - AT: {file_path}")
    except Exception as e:
        print(f"ERROR - COULD NOT SAVE {url} - {e}")

In [ ]:
import os
import io

def saveInDestFolder(searchNames,destDir,totalImgs,driver):
    for name in list(searchNames):
        # path=os.path.join(destDir,name)
        # if not os.path.isdir(path):
        #     os.mkdir(path)
        print('Current Path',destDir)
        totalLinks=getImageUrls(name,totalImgs,driver)
        print('totalLinks',totalLinks)

        if totalLinks is None:
            print('images not found for :',name)
            continue
        else:
            for i, link in enumerate(totalLinks):
                file_name = f"{i:150}.jpg"
                downloadImages(destDir,file_name,link)
            

In [ ]:
searchNames=['wear glasses'] 
destDir=f'/content/drive/My Drive/Glass/images'
totalImgs=3000

saveInDestFolder(searchNames,destDir,totalImgs,wd)

Current Path /content/drive/My Drive/Glass/images
Found: 100 search results. Extracting links from 0:100
Found: 13 looking for more image links ...
Found: 24 looking for more image links ...
Found: 37 looking for more image links ...
Found: 50 looking for more image links ...
Found: 63 looking for more image links ...
Found: 76 looking for more image links ...
Found: 89 looking for more image links ...
Found: 100 looking for more image links ...
Found: 113 looking for more image links ...
Found: 126 looking for more image links ...
Found: 139 looking for more image links ...
Found: 152 looking for more image links ...
Found: 154 looking for more image links ...
Found: 158 looking for more image links ...
Found: 171 looking for more image links ...
Found: 183 looking for more image links ...
Found: 194 looking for more image links ...
Found: 198 looking for more image links ...
Found: 210 looking for more image links ...
Found: 224 looking for more image links ...
Found: 237 looking for